In [1]:
import os
import sys
import logging
import numpy as np
import pandas as pd
import urlparse


os.sys.path.append("/home/sparrow/sparrow/malicious_url_cluster")
from logger import logger 
from config import cfg

"""
参数设置 0.3 0.3 kmeans
total cluster num:	618
big cluster:	183
small cluster:	175
single one:	260
regex extract 346

参数设置 0.3 0.3 hier
total cluster num:	620
big cluster:	185
small cluster:	178
single one:	257
regex extract 352
"""
cfg.N_CLUSTER_RATION = 200

cfg.EDIT_DISTANCE_THRESH_LONG = 0.3
cfg.EDIT_DISTANCE_THRESH_SHORT = 0.3
cfg.LONG_URL_THRESH = 80
cfg.SHORT_URL_THRESH = 20

cfg.MIN_SUBSTRING_SIZE = 12
cfg.SINGLE_REGEX_SIZE = 30
cfg.TOTAL_REGEX_SIZE = 30

from vectorize import make_vectorize
from cluster_engine import make_kmeans_cluster, make_string_distance_cluster, make_hcluster
from regex_engine import regex_extract, regex_check, regex_publish
import preprocess
import cluster_engine
import regex_engine

In [4]:
def data_loader(filepath, csv = True, txt = False):
    try:
        if csv:
            df = pd.read_csv(filepath)
            urls = list(df.url)
        if txt:
            with open(filepath, "r") as fd:
                urls = [_.strip() for _ in fd]
        logger.debug("Malware URL Count:\t%d" %len(urls))    
        return urls
    except Exception as e:
        logger.error("%s FILE OPEN ERROR! %s" %(filepath, str(e)))
        sys.exit(0)

In [3]:
urls = data_loader("../../data/VirusTotalData/VT20160725_benign", csv = False, txt = True)

2018-04-09 14:24:32 <ipython-input-2-3401d29b96cd> [line:9] DEBUG	Malware URL Count:	8103035


In [4]:
url_domain, url_path, url_param = preprocess.url_statistic_analysis(urls)
url_path_dict, url_path_counter = preprocess.url_path_statistic_analysis(url_path)

print "only domain", len(url_domain)
print "only path", len(url_path)
print "with params", len(url_param)
print url_path_counter

only domain 0
only path 2755166
with params 36423
{0: 1109114, 1: 144634, 2: 290889, 3: 363164, 4: 246319, 5: 164473, 6: 103303, 7: 67838, 8: 99387, 9: 34643, 10: 32313, 11: 13084, 12: 12324, 13: 5989, 14: 3160, 15: 4019, 16: 1727, 17: 1661, 18: 1657, 19: 1540, 20: 1506, 21: 1484, 22: 1396, 23: 1564, 24: 1754, 25: 1664, 26: 1663, 27: 1688, 28: 2017, 29: 2186, 30: 2255, 31: 2506, 32: 2965, 33: 3445, 34: 3558, 35: 3631, 36: 3545, 37: 3277, 38: 2892, 39: 2288, 40: 1623, 41: 1121, 42: 658, 43: 437, 44: 300, 45: 257, 46: 235, 47: 284, 48: 342, 49: 230, 50: 153, 51: 156, 52: 84, 53: 114, 54: 85, 55: 63, 56: 59, 57: 50, 58: 15, 59: 15, 60: 11, 61: 12, 62: 6, 63: 12, 64: 7, 65: 28, 66: 34, 67: 22, 68: 25, 69: 32, 70: 29, 71: 32, 72: 61, 73: 18, 74: 8, 75: 5, 76: 1, 78: 1, 79: 7, 81: 3, 83: 2, 85: 4, 86: 1, 87: 2}


In [5]:
urls = data_loader("../../data/VirusTotalData/VT20160725_malicious", csv = False, txt = True)

2018-04-09 14:41:08 <ipython-input-4-3401d29b96cd> [line:9] DEBUG	Malware URL Count:	940163


In [6]:
url_domain, url_path, url_param = preprocess.url_statistic_analysis(urls)
url_path_dict, url_path_counter = preprocess.url_path_statistic_analysis(url_path)

print "only domain", len(url_domain)
print "only path", len(url_path)
print "with params", len(url_param)
print url_path_counter

only domain 685
only path 852114
with params 2540
{0: 19810, 1: 135242, 2: 214694, 3: 185970, 4: 131382, 5: 61828, 6: 52139, 7: 17821, 8: 11436, 9: 4336, 10: 2575, 11: 2255, 12: 2748, 13: 947, 14: 675, 15: 661, 16: 779, 17: 802, 18: 695, 19: 750, 20: 844, 21: 769, 22: 581, 23: 560, 24: 172, 25: 137, 26: 147, 27: 120, 28: 107, 29: 116, 30: 82, 31: 71, 32: 69, 33: 55, 34: 53, 35: 51, 36: 35, 37: 57, 38: 51, 39: 37, 40: 37, 41: 43, 42: 27, 43: 39, 44: 18, 45: 15, 46: 10, 47: 7, 48: 9, 49: 4, 50: 9, 51: 8, 52: 11, 53: 10, 54: 7, 55: 7, 56: 9, 57: 8, 58: 4, 59: 7, 60: 9, 61: 6, 62: 7, 63: 6, 64: 3, 65: 8, 66: 4, 67: 8, 68: 2, 69: 8, 70: 4, 71: 8, 72: 5, 73: 6, 74: 6, 75: 5, 76: 5, 77: 7, 78: 4, 79: 5, 80: 6, 81: 7, 82: 5, 83: 5, 84: 4, 85: 14, 96: 1, 166: 1, 170: 5, 186: 1, 187: 1}


In [7]:
#preprocess.dump_urls(url_domain, "../../data/phishtank/phishtank_url_domain.csv")
#preprocess.dump_urls(url_path, "../../data/phishtank/phishtank_url_path.csv")
preprocess.dump_urls(url_param, "../../data/EXP_VirusTotal2/url_param.csv")

2018-04-09 14:41:32 preprocess.py [line:52] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_VirusTotal2/url_param.csv
2018-04-09 14:41:32 preprocess.py [line:56] DEBUG	URLs has been dump	../../data/EXP_VirusTotal2/url_param.csv


In [8]:
df = make_vectorize(url_param, domain = True, path = True, param = True,  output_path = "../../data/EXP_VirusTotal2/vector_param.csv")

2018-04-09 14:41:33 vectorize.py [line:93] DEBUG	vectorization complete! data shape:	(2540, 384)
2018-04-09 14:41:33 vectorize.py [line:38] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_VirusTotal2/vector_param.csv
2018-04-09 14:41:34 vectorize.py [line:41] DEBUG	vector has beeen dump	../../data/EXP_VirusTotal2/vector_param.csv


In [9]:
make_hcluster(data = df, output_path = '../../data/EXP_VirusTotal2/cluster_hier_param.json')

2018-04-09 14:41:41 cluster_engine.py [line:182] DEBUG	beggin to make hierarchical cluster, total_data_size: 2540 n_clusters: 12
2018-04-09 14:41:42 cluster_engine.py [line:196] DEBUG	hcluster done!
2018-04-09 14:41:42 cluster_engine.py [line:153] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_VirusTotal2/cluster_hier_param.json
2018-04-09 14:41:42 cluster_engine.py [line:157] DEBUG	K-means cluster has been dump	../../data/EXP_VirusTotal2/cluster_hier_param.json


In [10]:
cluster_engine.make_string_distance_cluster(data = '../../data/EXP_VirusTotal2/cluster_hier_param.json', 
                             metric = "distance",  
                             file_path = "../../data/EXP_VirusTotal2/cluster_distance_param.json")

2018-04-09 14:42:47 cluster_engine.py [line:79] DEBUG	K-means data has been loaded	../../data/EXP_VirusTotal2/cluster_hier_param.json
2018-04-09 14:42:47 cluster_engine.py [line:118] DEBUG	----------------0------------------
2018-04-09 14:42:47 cluster_engine.py [line:129] DEBUG	www.zurnaalem.net/sahsi-&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bcent%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAcirc%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3BAtilde%3B&amp%3BAtilde%3B&Atilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&amp%3BAcirc%3B&Atilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atil

2018-04-09 14:42:47 cluster_engine.py [line:129] DEBUG	www.zurnaalem.net/sahsi-&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bcent%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAcirc%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3BAtilde%3B&amp%3BAtilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&amp%3BAcirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&amp%3Bamp%3BAcirc%3B&amp%3BAtilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&amp%3BAcirc%

2018-04-09 14:42:48 cluster_engine.py [line:129] DEBUG	www.zurnaalem.net/sahsi-&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bcent%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAcirc%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3BAtilde%3B&amp%3Bamp%3BAtilde%3B&amp%3BAtilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&amp%3BAcirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&amp%3Bamp%3BAcirc%3B&amp%3BAtilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&amp%3BAcirc%3B&Atilde%3B%25C2%2582&Acirc%3

2018-04-09 14:42:48 cluster_engine.py [line:129] DEBUG	www.zurnaalem.net/sahsi-&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bcent%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAcirc%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3BAtilde%3B&amp%3BAtilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&amp%3BAcirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&amp%3Bamp%3BAcirc%3B&amp%3BAtilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&amp%3BAcirc%

2018-04-09 14:42:48 cluster_engine.py [line:129] DEBUG	www.zurnaalem.net/sahsi-&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bcent%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAcirc%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3BAtilde%3B&amp%3BAtilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&amp%3BAcirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&amp%3Bamp%3BAcirc%3B&amp%3BAtilde%3B&Ati

...

2018-04-09 14:46:18 cluster_engine.py [line:129] DEBUG	www.ba-inter.net/&amp%3BAtilde%3B&Atilde%3B&Atilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2597&amp%3BAcirc%3B&Atilde%3B&Atilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%259C&amp%3B

2018-04-09 14:46:18 cluster_engine.py [line:129] DEBUG	www.ba-inter.net/&amp%3Bamp%3BAtilde%3B&amp%3BAtilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&amp%3BAcirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2597&amp%3Bamp%3BAcirc%3B&amp%3BAtilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&amp%3BAcirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%259C&amp%3Bamp%3BAtilde%3B&amp%3BAtilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&amp%3BAcirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2597&amp%3Bamp%3BAcirc%3B&amp%3BAtilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&amp%3BAcirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B

2018-04-09 14:46:18 cluster_engine.py [line:129] DEBUG	www.ba-inter.net/&amp%3BAtilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2597&amp%3BAcirc%3B&amp%3Bbrvbar%3B&amp%3BAtilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2597&amp%3BAcirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2599&amp%3BAtilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2597&amp%3BAcirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2595&amp%3BAtilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2597&amp%3BAcirc%3B&amp%3Buml%3B&amp%3BAtilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2597&amp%3BAcirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde

2018-04-09 14:46:19 cluster_engine.py [line:129] DEBUG	www.ba-inter.net/&Atilde%3B&Atilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2597&Acirc%3B&Atilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%259C&Atilde%3B&Atilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2597&Acirc%3B&Atilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2599&Atilde%3B&Atilde%3B&Atilde%3B&Atilde%

2018-04-09 14:46:19 cluster_engine.py [line:129] DEBUG	www.ba-inter.net/&Atilde%3B&Atilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2597&Acirc%3B&ordf%3B&Atilde%3B&Atilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2597&Acirc%3B&Atilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2595&Atilde%3B&Atilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2597&Acirc%3B&Atilde%3B&

In [11]:
regex_extract(input_file_path = "../../data/EXP_VirusTotal2/cluster_distance_param.json", 
                  output_file_path = "../../data/EXP_VirusTotal2/regex_raw_param.txt")

2018-04-09 14:47:16 regex_engine.py [line:46] DEBUG	Cluster Data has been loaded	../../data/EXP_VirusTotal2/cluster_distance_param.json
2018-04-09 14:47:16 regex_engine.py [line:254] DEBUG	total cluster num:	620
2018-04-09 14:47:16 regex_engine.py [line:256] DEBUG	big cluster:	185
2018-04-09 14:47:16 regex_engine.py [line:258] DEBUG	small cluster:	178
2018-04-09 14:47:16 regex_engine.py [line:259] DEBUG	single one:	257
2018-04-09 14:47:16 regex_engine.py [line:260] DEBUG	cluster size detail:	Counter({1: 257, 2: 101, 3: 41, 5: 37, 4: 36, 7: 21, 6: 15, 11: 14, 9: 14, 10: 9, 8: 8, 13: 8, 12: 7, 15: 7, 19: 6, 16: 5, 14: 3, 18: 3, 17: 2, 20: 2, 23: 2, 27: 2, 57: 2, 137: 1, 21: 1, 22: 1, 25: 1, 26: 1, 28: 1, 31: 1, 34: 1, 36: 1, 37: 1, 38: 1, 170: 1, 130: 1, 88: 1, 89: 1, 91: 1, 230: 1, 42: 1})
2018-04-09 14:47:29 regex_engine.py [line:223] DEBUG	www.zurnaalem.net/sahsi-&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bcent%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bam

2018-04-09 14:47:29 regex_engine.py [line:223] DEBUG	www.zurnaalem.net/sahsi-&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bcent%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAcirc%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3Bamp%3BAtilde%3B&amp%3Bamp%3BAtilde%3B&amp%3BAtilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&amp%3BAcirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&amp%3Bamp%3BAcirc%3B&amp%3BAtilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&amp%3BAcirc%3B

2018-04-09 14:48:03 regex_engine.py [line:223] DEBUG	www.paimia.com/life/200912-paimia-&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bam
2018-04-09 14:48:03 regex_engine.py [line:223] DEBUG	blogs.fvawebsite.nl/matlabreadsegy/2011/01/19/&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp
2018-04-09 14:48:03 regex_engine.py [line:223] DEBUG	www.paimia.com/life/200912-paimia-&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bam

2018-04-09 14:48:04 regex_engine.py [line:223] DEBUG	www.paimia.com/life/200912-paimia-&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bam
2018-04-09 14:48:04 regex_engine.py [line:223] DEBUG	blogs.fvawebsite.nl/matlabreadsegy/2011/01/19/&amp%3Bamp%3Bamp%3Bamp%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Ba
2018-04-09 14:48:04 regex_engine.py [line:223] DEBUG	www.paimia.com/life/200912-paimia-&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3

2018-04-09 14:48:04 regex_engine.py [line:223] DEBUG	www.paimia.com/life/200912-paimia-&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Ba
2018-04-09 14:48:04 regex_engine.py [line:223] DEBUG	blogs.fvawebsite.nl/matlabreadsegy/2011/01/19/&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Batilde/%3B&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp/%3Bamp/%3Bamp
2018-04-09 14:48:04 regex_engine.py [line:223] DEBUG	blogs.fvawebsite.nl/matlabreadsegy/2011/01/19/&amp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Bamp/%3Ba

...

2018-04-09 14:58:12 regex_engine.py [line:223] DEBUG	www.ba-inter.net/&Atilde%3B&Atilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2597&Acirc%3B&ordf%3B&Atilde%3B&Atilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2597&Acirc%3B&Atilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2595&Atilde%3B&Atilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2597&Acirc%3B&Atilde%3B&At

2018-04-09 14:58:36 regex_engine.py [line:223] DEBUG	www.ba-inter.net/&Atilde%3B&Atilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2597&Acirc%3B&Atilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%259C&Atilde%3B&Atilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2597&Acirc%3B&Atilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2599&Atilde%3B&Atilde%3B&Atilde%3B&Atilde%3B

2018-04-09 14:58:36 regex_engine.py [line:223] DEBUG	www.ba-inter.net/&amp%3Bamp%3BAtilde%3B&amp%3BAtilde%3B&amp%3BAtilde%3B&amp%3BAtilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&amp%3BAcirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&amp%3BAcirc%3B&amp%3BAtilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&amp%3BAcirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&amp%3BAcirc%3B&amp%3BAtilde%3B&amp%3BAtilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&amp%3BAcirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&amp%3BAcirc%3B&amp%3BAtilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&amp%3BAcirc%3B&Atilde%3B&A

2018-04-09 14:58:49 regex_engine.py [line:223] DEBUG	www.ba-inter.net/&Atilde%3B&Atilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2597&Acirc%3B&brvbar%3B&Atilde%3B&Atilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2597&Acirc%3B&Atilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2599&Atilde%3B&Atilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2597&Acirc%3B&Atilde%3B&

2018-04-09 14:58:49 regex_engine.py [line:227] DEBUG	index:	4	['www\\.ba-inter\\.net/', '&amp%3BAtilde%3B&Atilde%3B&', 'Atilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&', 'Atilde%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&', '82&Acirc%3B&', 'Acirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%25', 'Acirc%3B&Atilde%3B&Atilde%3', 'Acirc%3B&Atilde%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2582&Acirc%3B&Atilde%3B%25C2%2582&Acirc%3B%25C2%2597&amp%3B']
2018-04-09 14:58:49 regex_engine.py [line:86] DEBUG	Regex has been dump	../../data/EXP_VirusTotal2/regex_raw_param.txt
2018-04-09 14:58:49 regex_engine.py [line:275] DEBUG	extract regex count:	352
2018-04-09 14:58:49 regex_engine.py [line:276] DEBUG	extract regex time cost:	692.898136


In [12]:
regex_check(input_file_path = "../../data/EXP_VirusTotal2/regex_raw_param.txt", 
            test_benign_file_path = "../../data/VirusTotalData/VT20160725_benign_plus.csv",
            test_malicious_file_path = "../../data/EXP_VirusTotal2/url_param.csv",
            result_file_path = "../../data/EXP_VirusTotal2/regex_result_param.txt", 
            n_jobs = 8)

2018-04-09 15:00:10 regex_engine.py [line:60] DEBUG	Regex Data has been loaded	../../data/EXP_VirusTotal2/regex_raw_param.txt
2018-04-09 15:00:14 regex_engine.py [line:71] DEBUG	Test Data has been loaded	../../data/VirusTotalData/VT20160725_benign_plus.csv
2018-04-09 15:00:14 regex_engine.py [line:71] DEBUG	Test Data has been loaded	../../data/EXP_VirusTotal2/url_param.csv


('batch index', 0, 'sample index', 0, 'FP', 0, 'TP', 38)
('batch index', 1, 'sample index', 0, 'FP', 0, 'TP', 5)
('batch index', 3, 'sample index', 0, 'FP', 0, 'TP', 28)
('batch index', 1, 'sample index', 1, 'FP', 0, 'TP', 17)
('batch index', 0, 'sample index', 1, 'FP', 0, 'TP', 34)
('batch index', 3, 'sample index', 1, 'FP', 0, 'TP', 6)
('batch index', 2, 'sample index', 0, 'FP', 0, 'TP', 22)
('batch index', 3, 'sample index', 2, 'FP', 0, 'TP', 230)
('batch index', 1, 'sample index', 2, 'FP', 0, 'TP', 239)
('batch index', 2, 'sample index', 1, 'FP', 0, 'TP', 2)
('batch index', 5, 'sample index', 0, 'FP', 0, 'TP', 2)
('batch index', 3, 'sample index', 3, 'FP', 0, 'TP', 48)
('batch index', 1, 'sample index', 3, 'FP', 0, 'TP', 321)
('batch index', 4, 'sample index', 0, 'FP', 0, 'TP', 2)
('batch index', 0, 'sample index', 2, 'FP', 0, 'TP', 29)
('batch index', 6, 'sample index', 0, 'FP', 0, 'TP', 3)
('batch index', 4, 'sample index', 1, 'FP', 0, 'TP', 53)
('batch index', 3, 'sample index',

('batch index', 5, 'sample index', 20, 'FP', 0, 'TP', 4)
('batch index', 3, 'sample index', 22, 'FP', 0, 'TP', 10)
('batch index', 7, 'sample index', 22, 'FP', 0, 'TP', 3)
('batch index', 6, 'sample index', 20, 'FP', 0, 'TP', 21)
('batch index', 5, 'sample index', 21, 'FP', 0, 'TP', 4)
('batch index', 1, 'sample index', 12, 'FP', 171, 'TP', 607)
('batch index', 4, 'sample index', 20, 'FP', 0, 'TP', 335)
('batch index', 7, 'sample index', 23, 'FP', 0, 'TP', 2)
('batch index', 5, 'sample index', 22, 'FP', 0, 'TP', 8)
('batch index', 6, 'sample index', 21, 'FP', 0, 'TP', 2)
('batch index', 7, 'sample index', 24, 'FP', 0, 'TP', 2)
('batch index', 3, 'sample index', 23, 'FP', 138, 'TP', 978)
('batch index', 2, 'sample index', 16, 'FP', 0, 'TP', 9)
('batch index', 5, 'sample index', 23, 'FP', 0, 'TP', 231)
('batch index', 4, 'sample index', 21, 'FP', 0, 'TP', 3)
('batch index', 6, 'sample index', 22, 'FP', 0, 'TP', 6)
('batch index', 3, 'sample index', 24, 'FP', 0, 'TP', 2)
('batch index', 0

('batch index', 2, 'sample index', 30, 'FP', 0, 'TP', 2)
('batch index', 1, 'sample index', 24, 'FP', 0, 'TP', 5)
('batch index', 0, 'sample index', 22, 'FP', 0, 'TP', 17)
('batch index', 2, 'sample index', 31, 'FP', 0, 'TP', 5)
('batch index', 7, 'sample index', 35, 'FP', 0, 'TP', 1)
('batch index', 1, 'sample index', 25, 'FP', 0, 'TP', 111)
('batch index', 7, 'sample index', 36, 'FP', 23, 'TP', 29)
('batch index', 2, 'sample index', 32, 'FP', 0, 'TP', 80)
('batch index', 1, 'sample index', 26, 'FP', 0, 'TP', 2)
('batch index', 2, 'sample index', 33, 'FP', 0, 'TP', 10)
('batch index', 2, 'sample index', 34, 'FP', 0, 'TP', 82)
('batch index', 0, 'sample index', 23, 'FP', 0, 'TP', 284)
('batch index', 1, 'sample index', 27, 'FP', 0, 'TP', 3)
('batch index', 2, 'sample index', 35, 'FP', 0, 'TP', 6)
('batch index', 7, 'sample index', 37, 'FP', 59, 'TP', 601)
('batch index', 2, 'sample index', 36, 'FP', 0, 'TP', 8)
('batch index', 2, 'sample index', 37, 'FP', 0, 'TP', 46)
('batch index', 1

2018-04-09 15:12:34 regex_engine.py [line:323] DEBUG	Check Result has been dump	../../data/EXP_VirusTotal2/regex_result_param.txt


In [9]:
regex_publish(result_file_path= "../../data/EXP_VirusTotal2/regex_result_param.txt",
              publish_file_path="../../data/EXP_VirusTotal2/regex_publish_param.txt",
              publish_fp_thresh = 0,
              publish_tp_thresh = 1)

2018-04-08 18:50:55 regex_engine.py [line:314] DEBUG	check Data has been loaded	../../data/VT/regex_result_param.txt
2018-04-08 18:50:55 regex_engine.py [line:328] DEBUG	Publish Result Count:	346
2018-04-08 18:50:55 regex_engine.py [line:329] DEBUG	Publish Result has been dump	../../data/VT/regex_publish_param.txt
